In [57]:
xmlNs = {"cmmn": "http://www.omg.org/spec/CMMN/20151109/MODEL"}
xmlPath = "/Users/wvw/Dropbox/research/projects/Montfort PM/cmmn/cap2.cmmn"

from rdflib import Namespace
modelNs = Namespace("http://ontario.org/qbp/cap#")
ttlPath = 'ttl/cap0.ttl'

# Parse

In [58]:
import xml.etree.ElementTree as ET
import pprint

def parse(path, ns):
    tree = ET.parse(path)
    for prefix, uri in ns.items():
        ET.register_namespace(prefix, uri)
    root = tree.getroot()
    
    itemObjs = {}
    planItems = root.findall(".//cmmn:planItem", namespaces=ns)

    for planItem in planItems:
        defRef = planItem.attrib['definitionRef']
        
        itemObj = {
            'sentries': {
                'entry': [],
                'exit': []
            }
        }
        itemObjs[defRef] = itemObj
        
        reqRules = planItem.findall(".//cmmn:itemControl/cmmn:requiredRule", namespaces=ns)
        repRules = planItem.findall(".//cmmn:itemControl/cmmn:repetitionRule", namespaces=ns)
        mandatory = len(reqRules) > 0; repetition = len(repRules) > 0
        itemObj['mandatory'] = mandatory
        itemObj['repetition'] = repetition
        
        label = None; typ = None
        if defRef.startswith("Task"):
            labelNode = root.findall(f".//cmmn:task[@id='{defRef}']", namespaces=ns)[0]
            label = labelNode.attrib['name']
            typ = 'Task'
        elif defRef.startswith("Stage"):
            stage = root.findall(f".//cmmn:stage[@id='{defRef}']", namespaces=ns)[0]
            label = stage.attrib['name']
            itemObj['children'] = []
            for childPlanItem in stage.findall(".//cmmn:planItem", namespaces=ns):
                itemObj['children'].append(childPlanItem.attrib['definitionRef'])
            typ = 'Stage'
        elif defRef.startswith("Milestone"):
            milestone = root.findall(f".//cmmn:milestone[@id='{defRef}']", namespaces=ns)[0]
            label = milestone.attrib['name']
            typ = 'Milestone'
        else:
            continue
            
        itemObj['label'] = label
        itemObj['type'] = typ
        
        itemObj['states'] = [ ( 'Inactive', 'init' ) ]
            
        # print(">", label, ("(mandatory)" if mandatory else ""))
        
        for entryCrit in planItem.findall("cmmn:entryCriterion", namespaces=ns): 
            # print("- sentry")              
            sentry = root.findall(f".//cmmn:sentry[@id='{entryCrit.attrib['sentryRef']}']", namespaces=ns)[0]
            
            sentryObj = {
                'id': sentry.attrib['id'],
                'items': [],
                'conditions': []
            }
            itemObj['sentries']['entry'].append(sentryObj)
            
            itemParts = sentry.findall(".//cmmn:planItemOnPart", namespaces=ns)
            for itemPart in itemParts:
                sentryItemObj = { 'id': itemPart.attrib['id'] }
                sourcePlanItem = root.findall(f".//cmmn:planItem[@id='{itemPart.attrib['sourceRef']}']", namespaces=ns)[0]
                sourceDefRef = sourcePlanItem.attrib['definitionRef']
                sentryItemObj['source'] = sourceDefRef
                # print("source:", sourceDefRef)
                
                events = itemPart.findall(".//cmmn:standardEvent", namespaces=ns)
                if len(events) > 0:
                    eventLabel = events[0].text
                    sentryItemObj['event'] = eventLabel
                    # print("event:", eventLabel)
            
                sentryObj['items'].append(sentryItemObj)
            
            associations = root.findall(f".//cmmn:association[@sourceRef='{entryCrit.attrib['id']}']", namespaces=ns)
            for association in associations:
                condItemObj = { 'id': association.attrib['targetRef'] }
                textAnnotations = root.findall(f".//cmmn:textAnnotation[@id='{association.attrib['targetRef']}']", namespaces=ns)
                if len(textAnnotations) > 0:
                    textAnnotation = textAnnotations[0].findall(".//cmmn:text", namespaces=ns)[0].text
                    # print("condition:", textAnnotation)
                    condItemObj['text'] = textAnnotation
                    sentryObj['conditions'].append(condItemObj)
                    
        # print("")
    
    return itemObjs

In [59]:
itemObjs = parse(xmlPath, xmlNs)
pprint.pprint(itemObjs)

{'Milestone_0gl7st9': {'label': 'hospital admission',
                       'mandatory': False,
                       'repetition': False,
                       'sentries': {'entry': [], 'exit': []},
                       'states': [('Inactive', 'init')],
                       'type': 'Milestone'},
 'Stage_1sveclw': {'children': ['Task_0kvjrmn',
                                'Task_0hsxzhz',
                                'Task_0rwjaka',
                                'Task_0akqm8t',
                                'Task_022k641'],
                   'label': '',
                   'mandatory': False,
                   'repetition': False,
                   'sentries': {'entry': [{'conditions': [],
                                           'id': 'Sentry_1a74myc',
                                           'items': [{'event': 'occur',
                                                      'id': 'PlanItemOnPart_1y7o923',
                                                      'so

# Convert

In [60]:
from rdflib import Literal, Graph, BNode, RDF, RDFS, XSD, URIRef
from rdflib.collection import Collection
from util import rdf_coll
from convert_base import str_to_uri

CM = Namespace("http://rdf.org/cmmn#")
ST = Namespace("http://rdf.org/state#")

def convert(itemObjs, modelNs, dest):
    g = Graph()

    for defRef, itemObj in itemObjs.items():
        planItemUri = str_to_uri(defRef, modelNs)

        if itemObj['label'].strip() != "":
            g.add((planItemUri, RDFS['label'], Literal(itemObj['label'])))
        
        g.add((planItemUri, CM['isMandatory'], Literal(itemObj['mandatory'], datatype=XSD['boolean'])))
        g.add((planItemUri, CM['hasRepetition'], Literal(itemObj['repetition'], datatype=XSD['boolean'])))

        g.add((planItemUri, RDF['type'], CM['PlanItem']))    
        g.add((planItemUri, RDF['type'], CM[itemObj['type']]))
        if itemObj['type'] == 'Stage':
            for child in itemObj['children']:
                g.add((planItemUri, CM['hasChild'], str_to_uri(child, modelNs)))
        
        curState = itemObj['states'][-1]
        g.add((planItemUri, ST['in'], rdf_coll(g, ST[curState[0]], Literal(curState[1]))))
                
        for sentry in itemObj['sentries']['entry']:
            sentryUri = str_to_uri(sentry['id'], modelNs)
            g.add((planItemUri, CM['hasSentry'], sentryUri))
            g.add((sentryUri, RDF['type'], CM['Sentry']))
            g.add((sentryUri, RDF['type'], CM['EntrySentry']))
            
            for item in sentry['items']:
                planItemPartUri = str_to_uri(item['id'], modelNs)
                g.add((sentryUri, CM['hasPlanItemPart'], planItemPartUri))
                
                sourceUri = str_to_uri(item['source'], modelNs)
                g.add((planItemPartUri, CM['hasSource'], sourceUri))
                
                eventLabel = item['event']
                g.add((planItemPartUri, CM['hasEvent'], Literal(eventLabel)))
                
            for condition in sentry['conditions']:
                conditionUri = str_to_uri(condition['id'], modelNs)
                g.add((sentryUri, CM['hasCondition'], conditionUri))
                
                g.add((conditionUri, RDFS['comment'], Literal(condition['text'])))
                
    g.serialize(format="n3", destination=dest)

In [62]:
convert(itemObjs, modelNs, "ttl/cap/model.ttl")

# Reason

## Steps in Python

In [5]:
from util import run
import re

def step(itemObjs, modelNs, dataPath, nr):
    target = dataPath[0:dataPath.index(".")] + str(nr) + dataPath[dataPath.index("."):]
        
    convert(itemObjs, modelNs, target)
    out = reason(target)
    update(itemObjs, out, modelNs)

    for defRef, itemObj in itemObjs.items():
        print(itemObj['label'], f"({defRef})", itemObj['states'])

def reason(dataPath):
    out = run(['eye', 'n3/state.n3', 'n3/workflow.n3', dataPath, '--nope', '--pass-only-new'])
    return out

def update(itemObjs, out, modelNs):    
    nsStr = modelNs.__str__()
    for defRef, state, reason, id in re.findall(f"<{nsStr}(.*?)> state:in \(\s*state:(.*?) \"(.*?)\" (\d+)\s*\)", out):
        itemObjs[defRef]['states'].append((state, reason, id))

In [6]:
# initialize

import copy
itemObjs2 = copy.deepcopy(itemObjs)

step(itemObjs2, modelNs, ttlPath, 0)

x-ray (Task_0u6b4ee) [('Inactive', 'init', 0), ('Ready', 'notInStage', '1'), ('Active', 'noSentries', '2')]
antibiotics (Task_10t4i0s) [('Inactive', 'init', 0), ('Ready', 'notInStage', '1')]
 (Stage_1sveclw) [('Inactive', 'init', 0), ('Ready', 'notInStage', '1')]
hospital admission (Milestone_0gl7st9) [('Inactive', 'init', 0), ('Ready', 'notInStage', '1'), ('Active', 'noSentries', '2')]
urine antigen testing (Task_1dy8o18) [('Inactive', 'init', 0), ('Ready', 'notInStage', '1')]
PCR for legionella (Task_1vk3nm4) [('Inactive', 'init', 0), ('Ready', 'notInStage', '1')]
Blood cultures (Task_1hoczvg) [('Inactive', 'init', 0), ('Ready', 'notInStage', '1')]
sputum culture (Task_0kvjrmn) [('Inactive', 'init', 0)]
gram staining (Task_0hsxzhz) [('Inactive', 'init', 0)]
Blood gases, lactate, liver enzyme and liver function tests (Task_0rwjaka) [('Inactive', 'init', 0)]
routine bloodwork: CBC, electrolytes, renal function (Task_0akqm8t) [('Inactive', 'init', 0)]
oxygen saturation (Task_022k641) [(

In [7]:
# first observation
states = itemObjs2['Milestone_0gl7st9']['states']
states.append(('Occurred', 'observation', len(states)))

In [8]:
step(itemObjs2, modelNs, ttlPath, 1)

x-ray (Task_0u6b4ee) [('Inactive', 'init', 0), ('Ready', 'notInStage', '1'), ('Active', 'noSentries', '2')]
antibiotics (Task_10t4i0s) [('Inactive', 'init', 0), ('Ready', 'notInStage', '1')]
 (Stage_1sveclw) [('Inactive', 'init', 0), ('Ready', 'notInStage', '1'), ('Active', 'sentryFired', '2')]
hospital admission (Milestone_0gl7st9) [('Inactive', 'init', 0), ('Ready', 'notInStage', '1'), ('Active', 'noSentries', '2'), ('Occurred', 'observation', 3)]
urine antigen testing (Task_1dy8o18) [('Inactive', 'init', 0), ('Ready', 'notInStage', '1')]
PCR for legionella (Task_1vk3nm4) [('Inactive', 'init', 0), ('Ready', 'notInStage', '1')]
Blood cultures (Task_1hoczvg) [('Inactive', 'init', 0), ('Ready', 'notInStage', '1')]
sputum culture (Task_0kvjrmn) [('Inactive', 'init', 0), ('Ready', 'inActiveStage', '1')]
gram staining (Task_0hsxzhz) [('Inactive', 'init', 0), ('Ready', 'inActiveStage', '1')]
Blood gases, lactate, liver enzyme and liver function tests (Task_0rwjaka) [('Inactive', 'init', 0),

## Steps in N3

### log:conclusion

In [ ]:
from util import run 

out = run(['eye', 'n3/concl/state.n3', 'n3/concl/allinone.n3', '--nope', '--pass-only-new'], printerr=True)
print(out)

time: 0.3283462079707533
error: eye --quiet n3/state.n3 n3/allinone.n3 --nope --pass-only-new
EYE v11.23.2 (2025-12-09)
SWI-Prolog version 9.2.7
starting 28 [msec cputime] 31 [msec walltime]
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/n3/state.n3 SC=4
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/n3/allinone.n3 SC=6
networking 4 [msec cputime] 5 [msec walltime]
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/cap.ttl SC=116
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/n3/state.n3 SC=4
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/n3/workflow.n3 SC=4
eye --quiet --nope /tmp/swipl_eye_28862_6 --pass-all
EYE v11.23.2 (2025-12-09)
SWI-Prolog version 9.2.7
starting 25 [msec cputime] 28 [msec walltime]
GET file:///tmp/swipl_eye_28862_6 SC=124
networking 10 [msec cputime] 10 [msec walltime]
reasoning 8 [msec cputime] 9 [msec walltime]
2026-01-25T22:42:52.888Z in=124 out=133 ent=142 step=654 brake=5 inf=306645 sec=0.043 inf/sec=7131279

GET file:///tmp/swipl_eye_28862_7 SC=133
eye --quiet --nope /tmp/sw

In [ ]:
from util import run 

out = run(['eye', 'n3/concl/test.n3', '--nope', '--pass-only-new'], printerr=True)
print(out)

out = run(['npx', 'eyeling', 'n3/concl/test.n3', '--nope', '--pass-only-new'], printerr=True)
print(out)

time: 0.19328604196198285
error: eye --quiet n3/test.n3 --nope --pass-only-new
EYE v11.23.2 (2025-12-09)
SWI-Prolog version 9.2.7
starting 26 [msec cputime] 34 [msec walltime]
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/n3/test.n3 SC=2
networking 1 [msec cputime] 1 [msec walltime]
eye --quiet --nope /tmp/swipl_eye_69093_2 --pass-all
EYE v11.23.2 (2025-12-09)
SWI-Prolog version 9.2.7
starting 25 [msec cputime] 28 [msec walltime]
GET file:///tmp/swipl_eye_69093_2 SC=2
networking 1 [msec cputime] 1 [msec walltime]
reasoning 1 [msec cputime] 0 [msec walltime]
2026-01-25T23:54:36.172Z in=2 out=3 ent=4 step=8 brake=2 inf=28094 sec=0.027 inf/sec=1040519

GET file:///tmp/swipl_eye_69093_3 SC=3
eye --quiet --nope /tmp/swipl_eye_69093_5 --pass-all
EYE v11.23.2 (2025-12-09)
SWI-Prolog version 9.2.7
starting 25 [msec cputime] 27 [msec walltime]
GET file:///tmp/swipl_eye_69093_5 SC=2
networking 1 [msec cputime] 1 [msec walltime]
reasoning 0 [msec cputime] 0 [msec walltime]
2026-01-25T23:54:36.213Z 

### N3 loop

In [ ]:
from util import run 

out = run(['eye', 'ttl/cap/model.ttl', 'ttl/cap/obs.ttl', 'n3/graph.n3', 'n3/state.n3', 'n3/workflow.n3', 'n3/run.n3', '--nope', '--pass-only-new'], printerr=True)
print(out)

time: 0.08521633315831423
error: eye --quiet ttl/cap/model.ttl ttl/cap/obs.ttl n3/graph.n3 n3/state.n3 n3/workflow.n3 n3/allinone.n3 --nope --pass-only-new
EYE v11.23.2 (2025-12-09)
SWI-Prolog version 9.2.7
starting 29 [msec cputime] 32 [msec walltime]
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/ttl/cap/model.ttl SC=128
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/ttl/cap/obs.ttl SC=1
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/n3/graph.n3 SC=1
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/n3/state.n3 SC=3
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/n3/workflow.n3 SC=17
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/n3/allinone.n3 SC=10
networking 18 [msec cputime] 20 [msec walltime]
reasoning 18 [msec cputime] 18 [msec walltime]
2026-01-29T16:11:59.082Z in=160 out=1 ent=2 step=4 brake=4 inf=564022 sec=0.065 inf/sec=8677262


# Processed by EYE v11.23.2 (2025-12-09)
# eye --quiet ttl/cap/model.ttl ttl/cap/obs.ttl n3/graph.n3 n3/state.n3 n3/workflow.n3 n3/allinone.n3 --nope --pass-only-new

@